# RFM-Analysis
Recency: How recently a customer has made a purchase

Frequency: How often a customer makes a purchase

Monetary Value: How much money a customer spends on purchases

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("data.csv", encoding='unicode_escape')

In [31]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Calculating M- monetary value

In [32]:
df['Total'] = df['UnitPrice']*df['Quantity']

In [33]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [34]:
m = df.groupby('CustomerID')['Total'].sum()
m = pd.DataFrame(m).reset_index()

In [35]:
m.head()

,CustomerID,Total
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


# Calculating F- frequency

In [36]:
freq = df.groupby('CustomerID')['InvoiceDate'].count()
f = pd.DataFrame(freq).reset_index()

In [37]:
f.head()

,CustomerID,InvoiceDate
0,12346.0,2
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


# Calculating R- recency

In [38]:
df['date'] = pd.to_datetime(df['InvoiceDate'])

In [39]:
df['rank'] = df.sort_values(['CustomerID','date']).groupby(['CustomerID'])['date'].rank(method='min').astype(int)

In [40]:
recent = df[df['rank']==1]

In [41]:
recent['recency'] = recent['date'] - pd.to_datetime('2010-12-01 08:26:00')

C:\Users\Tedd\AppData\Local\Temp\ipykernel_37988\3018059465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent['recency'] = recent['date'] - pd.to_datetime('2010-12-01 08:26:00')


In [42]:
recent.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,date,rank,recency
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,2010-12-01 08:26:00,1,0 days
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2010-12-01 08:26:00,1,0 days
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,2010-12-01 08:26:00,1,0 days
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2010-12-01 08:26:00,1,0 days
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2010-12-01 08:26:00,1,0 days


In [43]:
def recency(recency):
    res = str(recency).split(' ')[0]
    return(int(res))
recent['recency'] = recent['recency'].apply(recency)

C:\Users\Tedd\AppData\Local\Temp\ipykernel_37988\3049604467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent['recency'] = recent['recency'].apply(recency)


In [44]:
recent = recent[['CustomerID','recency']]
recent = recent.drop_duplicates()

# merging all dataframes

In [45]:
finaldf = f.merge(m,on='CustomerID').merge(recent,on='CustomerID')

In [46]:
finaldf.head()

,CustomerID,InvoiceDate,Total,recency
0,12346.0,2,0.00,48
1,12347.0,182,4310.00,6
2,12348.0,31,1797.24,15
3,12349.0,73,1757.55,355
4,12350.0,17,334.40,63
